In [142]:
import os
import pickle
import torch
import pandas as pd
import numpy as np
from torch.distributions import Categorical

# Fixed Split for Davidson

In [252]:
with open('../../data/davidson/annotated_tweets', 'rb') as file_tweets:
    annotated_tweets = pickle.load(file_tweets)

In [344]:
df = torch.load('../../data/davidson/User_overview.pt')
df

,User,class_0,class_1,class_2,Total,Following,Follower,Network,Category,Category_joined
0,27511025,3,137,7,147,2,2,1,3,3
1,1131735763,21,64,48,133,1,0,1,3,3
2,229536338,4,117,4,125,11,8,1,3,3
3,1703531550,5,110,5,120,2,7,1,3,3
4,200389104,5,93,7,105,8,5,1,3,3
...,...,...,...,...,...,...,...,...,...,...
6720,31190263,0,1,0,1,7,4,1,1,1
6721,218285161,0,1,0,1,13,12,1,1,1
6722,145063027,0,1,0,1,2,3,1,1,1
6723,413152043,0,1,0,1,1,0,1,1,1


In [198]:
def assign_users_to_train_val_test_DAVIDSON(df, splits = [0.6,0.2,0.2]):
    splits = torch.tensor(splits)
    train_users = []
    val_users = []
    test_users = []
    tweet_distributions = torch.zeros((3,8,3))
    for i in range(8):
        sub_df = df.loc[df['Category_joined'] == i]
        sizer = torch.tensor([len(sub_df)])
        sub_df.index = np.arange(sizer.item())
        samples = pd.DataFrame(Categorical(splits).sample(sizer)).astype("int")
        sub_df = pd.concat([sub_df,samples],axis = 1, join = 'inner')
        for user_run in sub_df.iterrows():
            if user_run[1][0] == 0:
                train_users.append(user_run[1]['User'])
                tweet_distributions[0][i][0] += user_run[1]['class_0']
                tweet_distributions[0][i][1] += user_run[1]['class_1']
                tweet_distributions[0][i][2] += user_run[1]['class_2']
            elif user_run[1][0] == 1:
                val_users.append(user_run[1]['User'])
                tweet_distributions[1][i][0] += user_run[1]['class_0']
                tweet_distributions[1][i][1] += user_run[1]['class_1']
                tweet_distributions[1][i][2] += user_run[1]['class_2']
            else:
                test_users.append(user_run[1]['User'])
                tweet_distributions[2][i][0] += user_run[1]['class_0']
                tweet_distributions[2][i][1] += user_run[1]['class_1']
                tweet_distributions[2][i][2] += user_run[1]['class_2']
    
    return train_users_d, val_users_d, test_users_d, tweet_distributions_d
        

In [235]:
train_users_d, val_users_d, test_users_d, tweet_distributions_d = assign_users_to_train_val_test(df)

In [243]:
len(train_users_d),len(val_users_d),len(test_users_d)

(4046, 1357, 1322)

In [244]:
tweet_distributions_d.sum()

tensor(14939.)

In [245]:
check_hate_train = tweet_distributions_d[0][:,0].sum()
check_hate_val = tweet_distributions_d[1][:,0].sum()
check_hate_test = tweet_distributions_d[2][:,0].sum()
check_overall_train = tweet_distributions_d[0,:,:].sum()
check_overall_val = tweet_distributions_d[1,:,:].sum()
check_overall_test = tweet_distributions_d[2,:,:].sum()
print(check_hate_train, check_hate_val, check_hate_test)
print(check_overall_train, check_overall_val, check_overall_test)

tensor(556.) tensor(125.) tensor(133.)
tensor(9423.) tensor(2748.) tensor(2768.)


In [238]:
tweet_distributions_d

tensor([[[  72.,    0.,    0.],
         [   0., 2119.,    0.],
         [   0.,    0.,  302.],
         [ 309., 3119.,  589.],
         [  73.,    0.,    0.],
         [   0., 1492.,    0.],
         [   0.,    0.,  266.],
         [ 102.,  673.,  307.]],

        [[  19.,    0.,    0.],
         [   0.,  709.,    0.],
         [   0.,    0.,  121.],
         [  42.,  578.,  117.],
         [  26.,    0.,    0.],
         [   0.,  519.,    0.],
         [   0.,    0.,   95.],
         [  38.,  376.,  108.]],

        [[  17.,    0.,    0.],
         [   0.,  747.,    0.],
         [   0.,    0.,   61.],
         [  43.,  656.,  183.],
         [  22.,    0.,    0.],
         [   0.,  486.,    0.],
         [   0.,    0.,   82.],
         [  51.,  326.,   94.]]])

In [281]:
# method to generate train, val and test tweets - DAVIDSON

tweets = annotated_tweets["text"].values.tolist()
labels = annotated_tweets["class"].values.tolist()
users = annotated_tweets["user"].values.tolist()
users = list(map(int, users))
length = len(tweets)
train_tweets, train_labels, train_users = [], [], []
val_tweets, val_labels, val_users = [], [], []
test_tweets, test_labels, test_users = [], [], []


s = dict()
for i in range(length):
    if users[i] in a:
        train_tweets.append(tweets[i])
        train_labels.append(labels[i])
        train_users.append(users[i])
    elif users[i] in b:
        val_tweets.append(tweets[i])
        val_labels.append(labels[i])
        val_users.append(users[i])
    elif users[i] in c:
        test_tweets.append(tweets[i])
        test_labels.append(labels[i])
        test_users.append(users[i])
    else:
        train_tweets.append(tweets[i])
        train_labels.append(labels[i])
        train_users.append(users[i])

print(len(train_labels),len(val_labels),len(test_labels))
    
s["train_tweets"], s["train_labels"], s["train_users"] = train_tweets, train_labels, train_users
s["val_tweets"], s["val_labels"], s["val_users"] = val_tweets, val_labels, val_users
s["test_tweets"], s["test_labels"], s["test_users"] = test_tweets, test_labels, test_users
        

9789 2550 2600


In [286]:
#torch.save(s, '../../data/davidson/fixed_split.pt')

# Fixed Splits for Waseem

In [295]:
annotated_tweets_waseem = torch.load('../../data/waseem/annotated_tweets.pt')

In [296]:
annotated_tweets_waseem

,labels,tweet_ids,user_ids,texts,labels_num,texts_preprocessed_ekphrasis,texts_preprocessed
0,racism,572342978255048705,2498963143,So Drasko just said he was impressed the girls...,1,"[so, drasko, just, said, he, was, impressed, t...","[drasko, -PRON-, impressed, girl, cook, half, ..."
1,racism,572341498827522049,110114783,Drasko they didn't cook half a bird you idiot ...,1,"[drasko, they, did, not, cook, half, a, bird, ...","[drasko, -PRON-, cook, half, bird, -PRON-, idi..."
2,racism,572340476503724032,38650214,Hopefully someone cooks Drasko in the next ep ...,1,"[hopefully, someone, cooks, drasko, in, the, n...","[hopefully, cook, drasko, ep, hashtag, mkr, ha..."
3,racism,572334712804384768,2587278392,of course you were born in serbia...you're as ...,1,"[of, course, you, were, born, in, serbia, ., <...","[course, -PRON-, bear, serbia, repeat, -PRON-,..."
4,racism,572332655397629952,2601524623,These girls are the equivalent of the irritati...,1,"[these, girls, are, the, equivalent, of, the, ...","[girl, equivalent, irritate, asian, girl, coup..."
...,...,...,...,...,...,...,...
16902,none,576359685843861505,2941145694,RT @JakeM_1998: RT BillSpindle: It's all about...,2,"[rt, <user>, :, rt, billspindle, :, it, ', s, ...","[rt, user, rt, billspindle, -PRON-, ', s, powe..."
16903,none,576612926838046720,2941145694,RT @ThinkAgain_DOS: Iraq: #ISIS sets off 21 ca...,2,"[rt, <user>, :, iraq, :, <hashtag>, isis, </ha...","[rt, user, iraq, hashtag, isis, hashtag, set, ..."
16904,none,576771329975664640,2941145694,RT @ThePatriot143: DEAR STATE DEPARTMENT: WHER...,2,"[rt, <user>, :, <allcaps>, dear, state, depart...","[rt, user, allcaps, dear, state, department, a..."
16905,none,560595245814267905,2756873076,"""@panelrific: Let's go 🐧🐧🐧🐧🐧🐧😃""",2,"["", <user>, :, let, us, go, 🐧, 🐧, 🐧, 🐧, 🐧, 🐧, ...","[ , user, let, -PRON-]"


In [298]:
df_waseem = torch.load('../../data/waseem/User_overview.pt')

,User,class_0,class_1,class_2,Total,Following,Follower,Network,Category,Category_joined
0,2941145694,51,1952,2543,4546,0,0,0,6,6
1,13857342,1,0,3853,3854,0,1,0,6,6
2,930620467,1371,8,969,2348,0,3,1,6,6
3,2756873076,1029,0,98,1127,5,3,0,6,6
4,545159350,4,0,47,51,4,7,1,2,2
...,...,...,...,...,...,...,...,...,...,...
2019,428909910,0,0,1,1,0,0,0,1,4
2020,20301438,0,0,1,1,0,0,0,1,4
2021,191741196,0,0,1,1,3,0,1,1,1
2022,26354418,0,0,1,1,4,4,1,1,1


In [299]:
def assign_users_to_train_val_test_WASEEM(df, splits = [0.6,0.2,0.2]):
    '''Difference to Davidson method: do not split category 6, this will be handled separately'''
    splits = torch.tensor(splits)
    train_users = []
    val_users = []
    test_users = []
    tweet_distributions = torch.zeros((3,6,3))
    for i in range(6):
        sub_df = df.loc[df['Category_joined'] == i]
        sizer = torch.tensor([len(sub_df)])
        sub_df.index = np.arange(sizer.item())
        samples = pd.DataFrame(Categorical(splits).sample(sizer)).astype("int")
        sub_df = pd.concat([sub_df,samples],axis = 1, join = 'inner')
        for user_run in sub_df.iterrows():
            if user_run[1][0] == 0:
                train_users.append(user_run[1]['User'])
                tweet_distributions[0][i][0] += user_run[1]['class_0']
                tweet_distributions[0][i][1] += user_run[1]['class_1']
                tweet_distributions[0][i][2] += user_run[1]['class_2']
            elif user_run[1][0] == 1:
                val_users.append(user_run[1]['User'])
                tweet_distributions[1][i][0] += user_run[1]['class_0']
                tweet_distributions[1][i][1] += user_run[1]['class_1']
                tweet_distributions[1][i][2] += user_run[1]['class_2']
            else:
                test_users.append(user_run[1]['User'])
                tweet_distributions[2][i][0] += user_run[1]['class_0']
                tweet_distributions[2][i][1] += user_run[1]['class_1']
                tweet_distributions[2][i][2] += user_run[1]['class_2']
    
    return train_users, val_users, test_users, tweet_distributions
        

In [300]:
train_users_w, val_users_w, test_users_w, tweet_distributions_w = assign_users_to_train_val_test_WASEEM(df_waseem)

In [304]:
print(len(train_users_w),len(val_users_w),len(test_users_w), sum((len(train_users_w),len(val_users_w),len(test_users_w))))

1180 424 416 2020


In [306]:
tweet_distributions_w.sum()

tensor(5032.)

In [329]:
# method to generate train, val and test tweets - Waseem
splits = torch.tensor([0.6,0.2,0.2])
tweets = annotated_tweets_waseem["texts"].values.tolist()
labels = annotated_tweets_waseem["labels_num"].values.tolist()
users = annotated_tweets_waseem["user_ids"].values.tolist()
users = list(map(int, users))
length = len(tweets)
train_tweets, train_labels, train_users = [], [], []
val_tweets, val_labels, val_users = [], [], []
test_tweets, test_labels, test_users = [], [], []
count_4users = 0

waseem_dict = dict()
for i in range(length):
    if users[i] in train_users_w:
        train_tweets.append(tweets[i])
        train_labels.append(labels[i])
        train_users.append(users[i])
    elif users[i] in val_users_w:
        val_tweets.append(tweets[i])
        val_labels.append(labels[i])
        val_users.append(users[i])
    elif users[i] in test_users_w:
        test_tweets.append(tweets[i])
        test_labels.append(labels[i])
        test_users.append(users[i])
    else :
        category = Categorical(splits).sample().item()
        #print(category)
        count_4users += 1
        if category == 0:
            train_tweets.append(tweets[i])
            train_labels.append(labels[i])
            train_users.append(users[i])
        elif category == 1:
            val_tweets.append(tweets[i])
            val_labels.append(labels[i])
            val_users.append(users[i])
        else: 
            test_tweets.append(tweets[i])
            test_labels.append(labels[i])
            test_users.append(users[i])

print(len(train_labels),len(val_labels),len(test_labels))
    
waseem_dict["train_tweets"], waseem_dict["train_labels"], waseem_dict["train_users"] = train_tweets, train_labels, train_users
waseem_dict["val_tweets"], waseem_dict["val_labels"], waseem_dict["val_users"] = val_tweets, val_labels, val_users
waseem_dict["test_tweets"], waseem_dict["test_labels"], waseem_dict["test_users"] = test_tweets, test_labels, test_users
        

10166 3424 3317


In [340]:
#torch.save(waseem_dict, '../../data/waseem/fixed_split.pt')

In [330]:
count_4users

11875

In [339]:
print("Train: ", train_labels.count(0),train_labels.count(1),train_labels.count(2))
print("Val: ", val_labels.count(0),val_labels.count(1),val_labels.count(2))
print("Test: ", test_labels.count(0), test_labels.count(1), test_labels.count(2))
#len(train_labels)

Train:  2049 1188 6929
Val:  699 385 2340
Test:  682 403 2232
